In [51]:
from composable import pipeable
from composable.strict import map
from composable import from_toolz as tlz
from composable.sequence import slice, head
from composable.strict import map, filter
from composable.string import replace, split
import re as r

In [5]:
with open('sell_short_trades_2.txt') as f:
    content = f.readlines()
content[:10]

['Symbol   Side    Cxl      Qty    Price       Bid        Ask        T-DatS-DatTradeID      TradeTiSS      Exbkr    TradeCommType  SourceCommission   Account   OrderID      GTL               Trailer Info               Clr\n',
 'CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS /17  /17  256945       00 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO\n',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX /17  /17  866087       04 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300\n',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX /17  /17  866096       04 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300\n',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX /17  /17  869780       23 0

In [23]:
fix_headers = pipeable(lambda s: s.replace('Trailer Info','TrailerInfo').replace('SourceCommission','Source Commission').replace('T-DatS-DatTradeID','T-Dat S-Dat TradeID').strip())
headers = content[0]
headers_fixed = (headers
>> fix_headers
)

In [24]:
headers_fixed

'Symbol   Side    Cxl      Qty    Price       Bid        Ask        T-Dat S-Dat TradeID      TradeTiSS      Exbkr    TradeCommType  Source Commission   Account   OrderID      GTL               TrailerInfo               Clr'

In [101]:
strip = pipeable(lambda s: s.strip())
no_commas = pipeable(lambda s: s.replace(',',''))

data_rows = content[1:]
trades = (data_rows
>> map(strip)
>> map(no_commas)
)

In [296]:
symbol_regex = r.compile(r'^(CERS|S)\s+.*')
get_symbol = pipeable(lambda s: symbol_regex.match(s).groups()[0])

symbol = (trades
>> map(get_symbol)
)
symbol[:5]

['CERS', 'CERS', 'CERS', 'CERS', 'CERS']

In [99]:
side_regex = r.compile(r'^(CERS|S)\s+(SS)\s+.*')
get_side = pipeable(lambda s: side_regex.match(s).groups()[1])

side = (trades
>> map(get_side)
)
side[:5]

['SS', 'SS', 'SS', 'SS', 'SS']

In [100]:
cxl_regex = r.compile(r'^(CERS|S)\s+(SS)\s+(NEW).*')
get_cxl = pipeable(lambda s: cxl_regex.match(s).groups()[2])

cxl = (trades
>> map(get_cxl)
)
cxl[:5]

['NEW', 'NEW', 'NEW', 'NEW', 'NEW']

In [103]:
qty_regex = r.compile(r'^(CERS|S)\s+(SS)\s+(NEW)\s+(\d+)\s+.*')
get_qty = pipeable(lambda s: qty_regex.match(s).groups()[3])

qty = (trades
>> map(get_qty)
)
qty[:5]

['2756', '100', '200', '100', '100']

In [110]:
price_regex = r.compile(r'^(CERS|S)\s+(SS)\s+(NEW)\s+(\d+)\s+(\d\.\d{6}).*')
get_price = pipeable(lambda s: price_regex.match(s).groups()[4])

price = (trades
>> map(get_price)
)
price[:5]

['2.400000', '2.360000', '2.350000', '2.350000', '2.350000']

In [111]:
bid_regex = r.compile(r'^(CERS|S)\s+(SS)\s+(NEW)\s+(\d+)\s+(\d\.\d{6}).*?(\d\.\d{6}).*')
get_bid = pipeable(lambda s: bid_regex.match(s).groups()[5])

bid = (trades
>> map(get_bid)
)
bid[:5]

['2.340000', '2.360000', '2.360000', '2.350000', '2.350000']

In [113]:
ask_regex = r.compile(r'^(CERS|S)\s+(SS)\s+(NEW)\s+(\d+)\s+(\d\.\d{6}).*?(\d\.\d{6})\s+(\d\.\d{6}).*')
get_ask = pipeable(lambda s: ask_regex.match(s).groups()[6])

ask = (trades
>> map(get_ask)
)
ask[:5]

['2.450000', '2.370000', '2.370000', '2.360000', '2.360000']

In [114]:
tdat_regex = r.compile(r'^(CERS|S)\s+(SS)\s+(NEW)\s+(\d+)\s+(\d\.\d{6}).*?(\d\.\d{6})\s+(\d\.\d{6})(\d\d/\d\d).*?(/17).*?')
get_tdat = pipeable(lambda s: tdat_regex.match(s).groups()[7] + tdat_regex.match(s).groups()[8])

tdat = (trades
>> map(get_tdat)
)
tdat[:5]

['06/14/17', '06/14/17', '06/14/17', '06/14/17', '06/14/17']

In [115]:
sdat_regex = r.compile(r'^(CERS|S)\s+(SS)\s+(NEW)\s+(\d+)\s+(\d\.\d{6}).*?(\d\.\d{6})\s+(\d\.\d{6})(\d\d/\d\d)(\d\d/\d\d).*?(/17).*?')
get_sdat = pipeable(lambda s: sdat_regex.match(s).groups()[8] + sdat_regex.match(s).groups()[9])

sdat = (trades
>> map(get_sdat)
)
sdat[:5]

['06/19/17', '06/19/17', '06/19/17', '06/19/17', '06/19/17']

In [117]:
tradeID_regex = r.compile(r'^(CERS|S)\s+(SS)\s+(NEW)\s+(\d+)\s+(\d\.\d{6}).*?(\d\.\d{6})\s+(\d\.\d{6})(\d\d/\d\d)(\d\d/\d\d)(\d+)\s.*')
get_tradeID = pipeable(lambda s: tradeID_regex.match(s).groups()[9])

tradeID = (trades
>> map(get_tradeID)
)
tradeID[:5]

['1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003']

In [124]:
tradetiss_regex = r.compile(r'^(CERS|S)\s+(SS)\s+(NEW)\s+(\d+)\s+(\d\.\d{6}).*?(\d\.\d{6})\s+(\d\.\d{6})(\d\d/\d\d)(\d\d/\d\d)(\d+)\s(.*?)\s.*')
get_tradetiss = pipeable(lambda s: tradetiss_regex.match(s).groups()[10])

tradeTiSS = (trades
>> map(get_tradetiss)
)
tradeTiSS[:5]

['09:30:CustSS',
 '10:20:ContraSSFREX',
 '10:20:ContraSSFREX',
 '10:20:ContraSSFREX',
 '10:22:ContraSSFREX']

In [260]:
exbkr_regex = r.compile(r'.*(\d{4})\sPerUnit.*?')
get_exbkr = pipeable(lambda s: exbkr_regex.match(s).groups()[0])

Exbkr = (trades
>> map(get_exbkr)
)
Exbkr[:5]

['0704', '0704', '0704', '0704', '0704']

In [128]:
tradecommtype_regex = r.compile(r'.*?(PerUnit).*?')
get_tradecommtype = pipeable(lambda s: tradecommtype_regex.match(s).groups()[0])

TradeCommType = (trades
>> map(get_tradecommtype)
)
TradeCommType[:5]

['PerUnit', 'PerUnit', 'PerUnit', 'PerUnit', 'PerUnit']

In [129]:
source_regex = r.compile(r'.*?PerUnit\s+(.*?)\s.*?')
get_source = pipeable(lambda s: source_regex.match(s).groups()[0])

Source = (trades
>> map(get_source)
)
Source[:5]

['TERM', 'TERM', 'TERM', 'TERM', 'TERM']

In [130]:
commission_regex = r.compile(r'.*?PerUnit\s+(.*?)\s(\d\.\d+)\s.*?')
get_commission = pipeable(lambda s: commission_regex.match(s).groups()[1])

Commission = (trades
>> map(get_commission)
)
Commission[:5]

['0.010000', '0.010000', '0.010000', '0.010000', '0.010000']

In [133]:
account_regex = r.compile(r'.*?PerUnit\s+(.*?)\s(\d\.\d+)\s+(.*?)\s.*?')
get_account = pipeable(lambda s: account_regex.match(s).groups()[2])

Account = (trades
>> map(get_account)
)
Account[:5]

['AABBC', 'AABBC', 'AABBC', 'AABBC', 'AABBC']

In [153]:
orderID_regex = r.compile(r'.*?PerUnit\s+(.*?)\s+(\d\.\d+)\s+(.*?)\s+(\D.*?)\s+(.*?)(CERS|MSCO|GSCO)')
get_orderID = pipeable(lambda s: orderID_regex.match(s).groups()[4] + orderID_regex.match(s).groups()[5])

orderID = (trades
>> map(get_orderID)
)
orderID[:5]

['1706149900003#STA=GSCO',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS']

In [154]:
gtl_regex = r.compile(r'.*?/17\s+/17\s+\d{6}\s+(\d{2}).*?')
get_gtl = pipeable(lambda s: gtl_regex.match(s).groups()[0])

gtl = (trades
>> map(get_gtl)
)
gtl[:5]

['00', '04', '04', '23', '29']

In [155]:
trailerinfo_regex = r.compile(r'.*?/17\s+/17\s+(\d{6})\s.*?')
get_trailerinfo = pipeable(lambda s: trailerinfo_regex.match(s).groups()[0])

TrailerInfo = (trades
>> map(get_trailerinfo)
)
TrailerInfo[:5]

['256945', '866087', '866096', '869780', '892888']

In [169]:
clr_regex = r.compile(r'.*?/17\s+/17\s+(\d{6})\s.*?')
get_clr = pipeable(lambda s: clr_regex.match(s).groups()[0])

Clr = (trades
>> map(get_clr)
)
Clr[:5]

['256945', '866087', '866096', '869780', '892888']

In [257]:
wrong_row_number_regex = r.compile(r'^(CERS|S)\s+(SS)\s+(NEW)\s+(\d+)\s+(\d\.\d{6})(....)(\d\.\d{6}).*')
wrong_row_number = pipeable(lambda s: wrong_row_number_regex.match(s).groups()[5])

wrong_row = (trades
>> map(wrong_row_number)
>> map(strip)
)
wrong_row[:5]

['', '1842', '', '', '']

In [188]:
updated_wrong_row = wrong_row[1:]
updated_wrong_row[:5]

['1842', '', '', '', '']

In [246]:
fixing_wrong_clr_1842 = pipeable(lambda s: (s + '00') if (s == '1842') else s)
fixing_wrong_clr_8543 = pipeable(lambda s: (s + '00') if (s == '8543') else s)
fixing_wrong_clr_4736 = pipeable(lambda s: (s + '40') if (s == '4736') else s)

fixed_wrong_clr = (updated_wrong_row
>> map(fixing_wrong_clr_1842)
>> map(fixing_wrong_clr_4736)
>> map(fixing_wrong_clr_8543)
)
fixed_wrong_clr[:5]
len(fixed_wrong_clr)

912

In [247]:
fixed_wrong_clr = fixed_wrong_clr + ['']
len(fixed_wrong_clr)

913

In [249]:
fixed_wrong_clr[:20]

['184200',
 '',
 '',
 '',
 '',
 '854300',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '854300',
 '',
 '',
 '',
 '',
 '',
 '']

In [254]:
clr_regex = r.compile(r'.*?(\d{6})$')
get_clr = pipeable(lambda s: (clr_regex.match(s).groups()[0]) if (clr_regex.match(s) != None) else '')

regular_Clr = (trades
>> map(get_clr)
)
regular_Clr[:20]


['',
 '854300',
 '854300',
 '854300',
 '854300',
 '',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300']

In [241]:
def combine_Clr(lista, listb):
    """
    
    """
    list_a = list(lista)
    list_b = list(listb)
    output = list(lista)

    i = 0
    while(i < len(list_a)):
        if len(list_a[i]) > 0:
            output[i] = list_a[i]
            #print(output[i] + 'a')
        else:
            output[i] = list_b[i]
            #print(output[i] + 'b')
        i = i + 1
    return output

In [242]:
combine_Clr(['','','ABC'], ['DEF','DEF','DEF'])

['DEF', 'DEF', 'ABC']

In [243]:
def test_combine_Clr():
    assert combine_Clr(['','','ABC'], ['DEF','DEF','DEF']) == ['DEF','DEF','ABC']
    assert combine_Clr(['','',''], ['DEF','DEF','DEF']) == ['DEF','DEF','DEF']
    assert combine_Clr(['ABC','ABC','ABC'], ['DEF','DEF','DEF']) == ['ABC','ABC','ABC']
    assert not combine_Clr(['ABC','ABC','ABC'], ['DEF','DEF','DEF']) == ['DEF','ABC','ABC']
test_combine_Clr()

In [258]:
Clr = combine_Clr(fixed_wrong_clr, regular_Clr)
Clr[:5]

['184200', '854300', '854300', '854300', '854300']

In [318]:
real_headers = headers_fixed.split(' ')
real_headers[:5]

['Symbol', '', '', 'Side', '']

In [288]:
def delete_empties(lista):
    """
    
    """
    list_a = list(lista)
    i = 0
    while i < len(list_a):
        if list_a[i] == '':
            del(list_a[i])
        else:
            i = i + 1
    return list_a

delete_empties(['','s','','s','',''])

['s', 's']

In [307]:
h2 = delete_empties(real_headers)
header = ','.join(h2) + '\n'
header

'Symbol,Side,Cxl,Qty,Price,Bid,Ask,T-Dat,S-Dat,TradeID,TradeTiSS,Exbkr,TradeCommType,Source,Commission,Account,OrderID,GTL,TrailerInfo,Clr\n'

In [297]:
data = [(symbol, side, cxl, qty, price, bid, ask, sdat, tdat, tradeID, tradeTiSS, Exbkr, TradeCommType, Source, Commission, Account, orderID, gtl, TrailerInfo, Clr) for symbol, side, cxl, qty, price, bid, ask, sdat, tdat, tradeID, tradeTiSS, Exbkr, TradeCommType, Source, Commission, Account, orderID, gtl, TrailerInfo, Clr in zip(symbol, side, cxl, qty, price, bid, ask, sdat, tdat, tradeID, tradeTiSS, Exbkr, TradeCommType, Source, Commission, Account, orderID, gtl, TrailerInfo, Clr)]
data[:1]

[('CERS',
  'SS',
  'NEW',
  '2756',
  '2.400000',
  '2.340000',
  '2.450000',
  '06/19/17',
  '06/14/17',
  '1706149900003',
  '09:30:CustSS',
  '0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC',
  '1706149900003#STA=GSCO',
  '00',
  '256945',
  '184200')]

In [309]:
join_data = pipeable(lambda s: ','.join(s))

almost_there = (data
>> map(join_data)
)
almost_there[:10]

['CERS,SS,NEW,2756,2.400000,2.340000,2.450000,06/19/17,06/14/17,1706149900003,09:30:CustSS,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=GSCO,00,256945,184200',
 'CERS,SS,NEW,100,2.360000,2.360000,2.370000,06/19/17,06/14/17,1706149900003,10:20:ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS,04,866087,854300',
 'CERS,SS,NEW,200,2.350000,2.360000,2.370000,06/19/17,06/14/17,1706149900003,10:20:ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS,04,866096,854300',
 'CERS,SS,NEW,100,2.350000,2.350000,2.360000,06/19/17,06/14/17,1706149900003,10:20:ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS,23,869780,854300',
 'CERS,SS,NEW,100,2.350000,2.350000,2.360000,06/19/17,06/14/17,1706149900003,10:22:ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS,29,892888,854300',
 'CERS,SS,NEW,100,2.360000,2.350000,2.360000,06/19/17,06/14/17,1706149900003,10:24:ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CER

In [316]:
closer = '\n'.join(almost_there)
output = header + closer

In [317]:
with open('sell_short_trades_final.csv', 'w') as outfile:
    outfile.write(output)